# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle

# INPUTS

In [2]:
# Paths
path_dados_trafego = os.path.join('..','dados','brutos','trafego-veiculos-eq_ANTT')
path_dados_derivados = os.path.join('..','dados','derivados','trafego_rodo_ANTT')

In [3]:
periodo = list(range(2010, 2025))

# Criando um dataframe consolidado com os dados de tráfego

In [4]:
# arquivos dos dados
arquivos = os.listdir(path_dados_trafego)

In [5]:
df_trafego = pd.DataFrame()
for ano in periodo:
    arquivo = [a for a in arquivos if a[:-4].endswith(str(ano))][0]
    df_ano = pd.read_csv(os.path.join(path_dados_trafego, arquivo), sep=';', encoding='cp1252', low_memory=False)
    df_trafego = pd.concat([df_trafego,df_ano])

# EDA Inicial

In [6]:
df_trafego.head()

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
0,RODOVIA DO AÇO,01-01-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,44146,1,44146,NaN
1,RODOVIA DO AÇO,01-01-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,35771,1,35771,NaN
2,RODOVIA DO AÇO,01-02-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,33455,1,33455,NaN
3,RODOVIA DO AÇO,01-02-2010,Crescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,27109,1,27109,NaN
4,RODOVIA DO AÇO,01-03-2010,Decrescente,"Praça 01 BR-393/RJ km 125,00",Categoria 1,Passeio,31677,1,31677,NaN


In [7]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 10 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   concessionaria              object
 1   mes_ano                     object
 2   sentido                     object
 3   praca                       object
 4   categoria                   object
 5   tipo_de_veiculo             object
 6   volume_total                object
 7   multiplicador_de_tarifa     object
 8   volume_veiculo_equivalente  object
 9   tipo_de_cobranca            object
dtypes: object(10)
memory usage: 251.8+ MB


In [8]:
for col in df_trafego:
    print(f"{col}: {df_trafego[col].isna().sum()}")

concessionaria: 0
mes_ano: 0
sentido: 0
praca: 0
categoria: 0
tipo_de_veiculo: 0
volume_total: 344
multiplicador_de_tarifa: 0
volume_veiculo_equivalente: 0
tipo_de_cobranca: 74780


In [9]:
df_trafego[df_trafego['volume_total'].isna()].sample(9)

,concessionaria,mes_ano,sentido,praca,categoria,tipo_de_veiculo,volume_total,multiplicador_de_tarifa,volume_veiculo_equivalente,tipo_de_cobranca
125542,ECOSUL,01/06/2023,Crescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
135524,RIOSP,01/06/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 8,Comercial,NaN,"6,00","0,00",N/I
135759,RIOSP,01/07/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria Esp. 07,Comercial,NaN,"7,00","0,00",N/I
125742,ECOSUL,01/08/2023,Crescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
125372,ECOSUL,01/04/2023,Decrescente,"Praça 04 BR-392/RS km 52,30",Categoria 9,Moto,NaN,"0,50","0,00",N/I
135286,RIOSP,01/05/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 2,Comercial,NaN,"2,00","0,00",N/I
125142,ECOSUL,01/02/2023,Crescente,"Praça 03 BR-116/RS km 541,20",Categoria 9,Moto,NaN,"0,50","0,00",N/I
136690,RIOSP,01/11/2023,Crescente,"Praça 08 BR-116/SP km 205,00",Categoria 2,Comercial,NaN,"2,00","0,00",N/I
121196,ECO050,01/12/2023,Crescente,"Praça 05 BR-050/MG km 104,90",Veículo Comercial Acima 10 eixos,Comercial,NaN,"10,00","0,00",N/I


In [10]:
df_trafego[df_trafego['volume_total'].isna()]['volume_veiculo_equivalente'].unique()

array(['0,00'], dtype=object)

In [11]:
df_trafego['mes_ano'].str[:2].unique()

array(['01', '14', '28', '31', '30', '02', '03', '04', '05', '06', '07',
       '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '29'], dtype=object)

In [12]:
df_trafego['mes_ano'].str[3:5].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [13]:
df_trafego['tipo_de_cobranca'].unique()

array([nan, 'N/I', 'Manual', 'Automática', 'Mista'], dtype=object)

In [14]:
df_trafego['volume_total'].isna().any()

True

In [15]:
df_trafego['volume_total'].isna().sum()

344

In [16]:
df_trafego['multiplicador_de_tarifa'].unique()

array(['1', '2', '1,5', '3', '4', '5', '6', '0,5', '7', '8', '9', '10',
       1.0, 2.0, 1.5, 3.0, 4.0, 5.0, 6.0, 0.5, 7.0, 8.0, 9.0, 10.0,
       '2,00', '3,00', '4,00', '5,00', '6,00', '0,50', '1,00', '1,50',
       '7,00', '8,00', '9,00', '10,00', '0,00', '11,00', '13,00', '15,00',
       '12,00', '14,00', '18,00', '20,00', '16,00', '17,00', '19,00'],
      dtype=object)

# Tratamento dos dados

In [17]:
# Colunas de ano, mes e dia
df_trafego['year'] = df_trafego['mes_ano'].str[-4:].astype('int') 
df_trafego['month'] = df_trafego['mes_ano'].str[3:5].astype('int')
df_trafego['day'] = df_trafego['mes_ano'].str[:2].astype('int') 

In [18]:
# Recriando a coluna de mes_ano, agora como datetime ao invés de string. Tem que ser em ingles pq a função do pandas exige
df_trafego['mes_ano'] = pd.to_datetime(df_trafego[['year', 'month', 'day']])

In [21]:
# preenchendo os NaNs, qdo aplicável:
df_trafego['volume_total'] = df_trafego['volume_total'].fillna(0) #o volume equivalente é zero sempre que volume_total é zero

In [22]:
# Convertendo os tipos de dados para numéricos:
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].apply(lambda x: x.split('.')[0] if type(x) == str else x)
df_trafego['volume_total'] = df_trafego['volume_total'].astype('int')

df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['volume_veiculo_equivalente'] = df_trafego['volume_veiculo_equivalente'].astype('float') # existe multiplicador de tarifa fracionário

In [23]:
# convertendo categoria para string (do contrário não salva para parquet):
df_trafego['categoria'] = df_trafego['categoria'].astype('string')

In [24]:
# convertendo o multiplicador de tarifa de string para decimal (p/manter a precisao):
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(lambda x: x.replace(',','.') if type(x) == str else x)
df_trafego['multiplicador_de_tarifa'] = df_trafego['multiplicador_de_tarifa'].apply(Decimal)

In [25]:
# O tipo de tráfego ora está em maisúcula, ora em minúscula.
# Colocando tudo p/maiúscula
df_trafego['tipo_de_veiculo'] = df_trafego['tipo_de_veiculo'].str.upper()

In [26]:
# algumas concessionárias estão ora em maiúsculas ora em minúsculas
# colocando tudo para maiúscula
df_trafego['concessionaria'] = df_trafego['concessionaria'].str.upper()

In [27]:
df_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000353 entries, 0 to 2329761
Data columns (total 13 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   concessionaria              object        
 1   mes_ano                     datetime64[ns]
 2   sentido                     object        
 3   praca                       object        
 4   categoria                   string        
 5   tipo_de_veiculo             object        
 6   volume_total                int64         
 7   multiplicador_de_tarifa     object        
 8   volume_veiculo_equivalente  float64       
 9   tipo_de_cobranca            object        
 10  year                        int64         
 11  month                       int64         
 12  day                         int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6), string(1)
memory usage: 320.5+ MB


In [30]:
df_trafego['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024])

In [32]:
df_trafego.query("year == 2024")['month'].unique()

array([1, 2, 3, 4, 5, 6])

# Criando coluna desambiguando categorias comercial x passeio

In [33]:
df_trafego['tipo_de_veiculo'].unique()

array(['PASSEIO', 'COMERCIAL', 'MOTO', 'VEÍCULO PEQUENO'], dtype=object)

In [34]:
df_trafego["TIPO_TRAFEGO"] = df_trafego['tipo_de_veiculo'].apply(lambda x: x if x == "COMERCIAL" else "PASSEIO")

In [35]:
df_trafego["TIPO_TRAFEGO"].unique()

array(['PASSEIO', 'COMERCIAL'], dtype=object)

# Criando colunas com trafégo em Veqs para comercial e passeio

In [36]:
df_trafego['VEQS_COMERCIAL'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'COMERCIAL',
    df_trafego['volume_veiculo_equivalente'],
    0
)
df_trafego['VEQS_PASSEIO'] = np.where(
    df_trafego['TIPO_TRAFEGO'] == 'PASSEIO',
    df_trafego['volume_veiculo_equivalente'],
    0
)

# Criando dataframe com o período inicial e final dos dados

In [37]:
df_periodos = df_trafego.groupby('concessionaria').agg(
    data_inicial=('mes_ano', 'min'),
    data_final=('mes_ano', 'max')
).reset_index()

In [38]:
df_periodos.sort_values(by='data_inicial', ascending=False)

,concessionaria,data_inicial,data_final
17,HOLDING DO SISTEMA RODOVIARIO RIO - SAO PAULO ...,2024-01-01,2024-06-03
25,VIA BRASIL,2023-02-01,2023-12-01
13,ECORIOMINAS,2022-10-01,2024-06-02
15,ECOVIAS DO ARAGUAIA,2022-10-01,2024-06-02
20,RIOSP,2022-03-01,2023-12-01
26,VIA COSTEIRA,2022-01-01,2024-04-01
16,ECOVIAS DO CERRADO,2020-11-14,2024-06-02
27,VIA SUL,2019-01-01,2024-05-03
18,MSVIA,2015-09-01,2024-05-03
8,CRO,2015-09-01,2024-06-03


# Agrupando por ano/concessionaria/tipo de trafego

In [39]:
df_trafego.columns

Index(['concessionaria', 'mes_ano', 'sentido', 'praca', 'categoria',
       'tipo_de_veiculo', 'volume_total', 'multiplicador_de_tarifa',
       'volume_veiculo_equivalente', 'tipo_de_cobranca', 'year', 'month',
       'day', 'TIPO_TRAFEGO', 'VEQS_COMERCIAL', 'VEQS_PASSEIO'],
      dtype='object')

In [40]:
cols_veqs = ['volume_veiculo_equivalente','VEQS_COMERCIAL','VEQS_PASSEIO']

In [42]:
dict_veqs = {}
for col in cols_veqs:
    dict_veqs[col] = df_trafego.pivot_table(
        index='concessionaria',
        columns='year',
        values=col,
        aggfunc='sum'
        ) 

# Salvando p/parquet

In [43]:
# dataframe total
df_trafego.to_parquet(os.path.join(path_dados_derivados,'df_trafego.parquet'))

In [44]:
# data inicial e final
df_periodos.to_parquet(os.path.join(path_dados_derivados,'df_periodos.parquet'))

In [45]:
for col in cols_veqs:
    print(col)

volume_veiculo_equivalente
VEQS_COMERCIAL
VEQS_PASSEIO


In [46]:
# dicionário com os dados de tráfego:
for veq in dict_veqs:
    dict_veqs[veq].to_parquet(os.path.join(path_dados_derivados, f'df_{veq}.parquet'))

# Salvando p/Excel

In [47]:
with pd.ExcelWriter(os.path.join(path_dados_derivados, 'dados_trafego_ANTT.xlsx')) as writer:
    df_periodos.to_excel(writer, sheet_name="periodo_concessionarias")
    for veq in dict_veqs:
        dict_veqs[veq].to_excel(writer, sheet_name=f"{veq}")